In [178]:
import pandas as pd

import numpy as np


In [179]:
predDf = pd.read_csv(
    "/home/alextay96/Desktop/new_workspace/DLDataPipeline/data/case_study/case_study_pred.csv"
)
gtDf = pd.read_csv(
    "/home/alextay96/Desktop/new_workspace/DLDataPipeline/data/case_study/case_study_gt.csv"
)
visionPredDf1 = pd.read_csv("/home/alextay96/Desktop/new_workspace/DLDataPipeline/data/results/saloon_v2/Saloon - 4 Dr_imgs_pred_output.csv")
visionPredDf2 = pd.read_csv("/home/alextay96/Desktop/new_workspace/DLDataPipeline/data/results/hatchback_v2/Hatchback - 5 Dr_imgs_pred_output.csv")
visionPredDf3 = pd.read_csv("/home/alextay96/Desktop/new_workspace/DLDataPipeline/data/results/suv_v2/SUV - 5 Dr_imgs_pred_output.csv")
visionPredDf = pd.concat([visionPredDf1, visionPredDf2,visionPredDf3 ])
majorParts = [x.replace("vision_", "") for x in  visionPredDf.columns]
majorParts

['Unnamed: 0',
 'CaseID',
 'bonnet',
 'bumper_front',
 'door_front_lh',
 'door_rear_lh',
 'engine',
 'fender_front_lh',
 'front_panel',
 'headlamp_lh',
 'non_external',
 'misc',
 'wheel',
 'windscreen_front',
 'bumper_rear',
 'door_front_rh',
 'door_rear_rh',
 'rear_compartment',
 'rear_panel',
 'rear_quarter_rh',
 'tail_lamp_rh',
 'fender_front_rh',
 'headlamp_rh',
 'rear_quarter_lh',
 'tail_lamp_lh',
 'grille']

In [180]:
len(predDf)

26

In [181]:
targetCols = [x for x in predDf.columns if "0" not in x and x != "CaseID"]
allRows = []
for c in targetCols:
    for caseId in predDf["CaseID"].unique().tolist():
        visionPred = -1
        if(c in majorParts):
            visionPred = visionPredDf[visionPredDf["CaseID"] == caseId][f"vision_{c}"].item()
            

        pred = predDf[predDf["CaseID"] == caseId][c].item()
        gt = gtDf[gtDf["CaseID"] == caseId][c].item()
        allRows.append(
            {
                "CaseID" : caseId,
                "part" : c,
                "v_pred":visionPred,
                "pred" : pred,
                "gt" : gt,
                
                "inspect" : None,

            }
        )
caseStudyInspectDf = pd.json_normalize(allRows)
caseStudyInspectDf.sort_values(by = ["CaseID", "part"], inplace=True)

In [182]:
caseStudyInspectDf

,CaseID,part,v_pred,pred,gt,inspect
269,10019013,air_conditioning,-1,1,1,None
449,10019013,airbag,-1,0,0,None
29,10019013,bonnet,1,1,1,None
59,10019013,bumper_front,1,1,1,None
359,10019013,bumper_rear,1,1,0,None
...,...,...,...,...,...,...
774,13278125,tail_lamp_rh,0,0,0,None
504,13278125,undercarriage,-1,0,0,None
534,13278125,wheel,0,0,0,None
804,13278125,windscreen_front,0,0,0,None


In [183]:
len(caseStudyInspectDf["part"].unique())

34

In [184]:
majorParts = 25 * 30
majorParts

750

In [185]:
154 / majorParts

0.20533333333333334

In [186]:
caseStudyWrong = caseStudyInspectDf[(caseStudyInspectDf["pred"] !=caseStudyInspectDf["gt"] )
                                    & (caseStudyInspectDf["v_pred"] != -1)
                                    
                                    ]
caseStudyWrong

,CaseID,part,v_pred,pred,gt,inspect
359,10019013,bumper_rear,1,1,0,None
419,10019013,engine,1,1,0,None
569,10019013,fender_front_lh,0,0,1,None
89,10019013,grille,0,0,1,None
389,10019013,misc,1,1,0,None
...,...,...,...,...,...,...
234,13278125,door_front_rh,0,0,1,None
414,13278125,engine,1,1,0,None
594,13278125,fender_front_rh,0,0,1,None
84,13278125,grille,0,0,1,None


In [187]:
len(caseStudyWrong) / len(caseStudyInspectDf)

0.15098039215686274

In [188]:

caseStudyWrongVis = caseStudyInspectDf[(caseStudyInspectDf["pred"] !=caseStudyInspectDf["v_pred"]) & (caseStudyInspectDf["v_pred"] != -1) ]
caseStudyWrongVis

,CaseID,part,v_pred,pred,gt,inspect
413,10281260,engine,1,0,0,None


In [203]:
# majorPartDf = caseStudyInspectDf[caseStudyInspectDf["v_pred"] != - 1]
majorPartDf = caseStudyInspectDf

In [204]:
majorPartDf["part"].unique()

array(['air_conditioning', 'airbag', 'bonnet', 'bumper_front',
       'bumper_rear', 'cooling_fan', 'door_front_lh', 'door_front_rh',
       'door_mirror_lh', 'door_mirror_rh', 'door_rear_lh', 'door_rear_rh',
       'engine', 'fender_front_lh', 'fender_front_rh', 'fog_lamp_lh',
       'fog_lamp_rh', 'front_panel', 'grille', 'headlamp_lh',
       'headlamp_rh', 'interior', 'misc', 'radiator', 'rear_compartment',
       'rear_panel', 'rear_quarter_lh', 'rear_quarter_rh', 'tail_lamp_lh',
       'tail_lamp_rh', 'undercarriage', 'wheel', 'windscreen_front',
       'windscreen_rear'], dtype=object)

In [205]:
fpr = len(majorPartDf[(majorPartDf["pred"]  == 1) & (majorPartDf["gt"]  == 0) ]) / len(majorPartDf[ (majorPartDf["gt"]  == 0)])
fpr

0.30828025477707005

In [206]:
fnr = len(majorPartDf[(majorPartDf["pred"]  == 0) & (majorPartDf["gt"]  == 1) ]) / len(majorPartDf[ (majorPartDf["gt"]  == 1)])
fnr

0.14893617021276595

In [207]:
acc = len(majorPartDf[(majorPartDf["pred"]  == majorPartDf["gt"])]) / len(majorPartDf)
acc

0.7284313725490196

In [208]:
allSubsetAcc = []
for caseId in majorPartDf["CaseID"].unique().tolist():
    rows = majorPartDf[majorPartDf["CaseID"] == caseId]
    correct = rows[rows["pred"] == rows["gt"]]
    subset_acc = len(correct) / len(rows)
    allSubsetAcc.append(subset_acc)
    
    # print(subset_acc)
print(np.mean(allSubsetAcc))

0.7284313725490196
